# Pulse Length for LiH is 84.05 ns with Flexible Partial Compilation

(should take max-per-slice over more runs though)

In [2]:
import sys
sys.path.append('../..')
import config
from fqc import uccsd, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [3]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [4]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 6000
decay =  max_iterations / 2
convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-12, 'update_step': 20}
reg_coeffs = {}

In [5]:
N = 4
connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)

In [6]:
circuit = uccsd.get_uccsd_circuit('LiH')

In [7]:
slices = uccsd.get_uccsd_slices(circuit, granularity=2, dependence_grouping=True)
print(len(slices))

8


(ran each of the 8 slices via parallel sbatch job array)

In [17]:
import os
times = []
for filename in os.listdir('LiH/'):
    if filename.endswith('.out'):
        lines = open('LiH/%s' % filename).readlines()
        if len(lines) > 5 and lines[-1].startswith('^^^SHORTEST TIME was '):
            result = lines[-1]
            time = float(result[len('^^^SHORTEST TIME was '):])
            if time > 50:
                print(time, filename)
            else:
                times.append(time)
            
print(times)
print(sum(times))

[3.0, 30.2, 3.0, 13.3, 13.3, 3.0, 3.5, 14.75]
84.05
